In [ ]:
from sensor_core import SensorManager
from sensor_core import strg_manager
from sensor_core import ser_manager
import numpy as np
from itertools import product

In [ ]:
# Find available serial ports
ser_manager.find_serial()

In [ ]:
# USER DEFINED PARAMS
ser_channel_key = ["red", "infrared", "violet"]                    # Names of serial channels
plot_channel_key = [["red", "infrared", "violet"]]                 # Names and gridplot arrangement of plot channels
commport = '/dev/ttyACM0'                                          # Name of comm port
baudrate = 115200                                                  # bits/second
window_size = 1                                                    # Number of points to acquire before updating plot

In [ ]:
# Custom Serial Acquisition Function
def acquire_data(ser):
    """ serial acquisition function handler. All parameters are internally generated
    Must include the following parameters:
    :param ser: serial port
    """
    window_size = 1
    num_channels = 3
    ser_data = np.zeros((window_size, num_channels))
    channel_data = np.array([])
    # Decode incoming data into ser_data array
    for i in product((range(window_size)), range(num_channels)):
        try:
            ser_data[i] = ser.readline().decode().strip()
        except ValueError:
            pass

    # If any zeros
    for i in range(window_size):
        if any(ser_data[i, :] == 0):
            pass
        else:
            channel_data = np.append(channel_data, ser_data[i][:])

    channel_data = np.reshape(channel_data, (int(len(channel_data) / num_channels), num_channels))

    return channel_data


In [ ]:
# Test custom serial function
# ser = ser_manager.SerialManager(commport=commport, baudrate=baudrate, num_channel=8)
# ser.setup_serial()
# ser.acquire_data(func=acquire_data)

In [ ]:
# Instantiate Sensor Manager class
sm = SensorManager(ser_channel_key=ser_channel_key,
                   plot_channel_key=plot_channel_key,
                   commport=commport, 
                   baudrate=baudrate, 
                   window_size = window_size)

In [ ]:
# Create 'update process' to handle acquired data
p1 = sm.update_data_process(save_data=True, 
                            filepath = "./serial_db.sqlite3",
                            func=acquire_data)
plot = sm.setup_plot()

In [ ]:
# Start update process, start plotting process
## Recommended to use in-built start_process function to avoid Windows OS issues
sm.start_process(p1)
plot.add_animations(sm.online_grid_plot_data)

In [ ]:
# Show plot
plot.show()